In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=50, suppress=True)
np.core.arrayprint._line_width = 250

# Primal Dual Method

Below is defined a function which comprises the Primal Dual Method that is similar to the Dual Simplex Method but the main difference is that this method doesn't requiere a initial dual feasible solution to be basic.

In [3]:
def primal_dual(A, b, c, w, variables, itera = 1):
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    #print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
    #      + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m, n = np.shape(A)
    
    wA = np.ravel(w @ A)
    Q = list()
    for i in range(len(c)):
        if c[i] == wA[i]:
            Q.append(i)
    
    #print(wA)
    print("\nQ =", [q + 1 for q in Q])
    
    # Defining Restricted Primal Problem
    print("\n\033[94mRestricted Primal Problem\033[0m")
    no_variables = len(variables)
    artificial_variables = ['x' + str(i) for i in range(no_variables + 1, no_variables + m + 1)]
    variables_RPP = [variables[i] for i in range(no_variables) if i in Q] + artificial_variables
    print("\nVariables RPP")
    print(variables_RPP)
    
    identity_matrix = np.eye(m, dtype = int)
    extracted_A = A[:, Q]
    A_RPP = np.concatenate((extracted_A, identity_matrix), axis = 1)
    print("\nMatrix A RPP")
    print(A_RPP)
    
    c_RPP = np.concatenate((np.zeros(len(Q), dtype=int), np.ones(m, dtype=int)), axis = 0)
    print("\nVector c RPP")
    print(c_RPP)
    
    m_RPP, n_RPP = np.shape(A_RPP)
    
    basic_indexes = [ind for ind in range(n_RPP - m_RPP, n_RPP)]
    non_basic_indexes = [ind for ind in range(n_RPP - m_RPP)]
    tableau = construct_tableau(A_RPP, b, c_RPP, basic_indexes)
    
    solution_RPP = simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables_RPP), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
    print("\nSolution RPP")
    print(solution_RPP)
    
    x0 = c_RPP.T@solution_RPP
    print("\nPerformance RPP")
    print(x0)
    
    if x0 == 0:
        
        remaining_variables = [variables[i] for i in range(no_variables) if variables[i] not in variables_RPP]
        remaining_values = np.zeros(len(remaining_variables), dtype=int)
        
        indexes_variables = [int(var[1:]) for var in variables_RPP + remaining_variables]
        dict_primal_solution = dict(zip(indexes_variables, np.concatenate((solution_RPP, remaining_values), axis = 0).tolist()))
        sorted_dict_primal_solution = dict(sorted(dict_primal_solution.items(), key = lambda x:x[0]))
        primal_solution_ls = list(sorted_dict_primal_solution.values())[:-len(Q)]
        primal_variables_ls = list(sorted_dict_primal_solution.keys())[:-len(Q)]
        primal_variables_ls = ['x' + str(i) for i in primal_variables_ls]
        results_primal_ls = [str(round(i, 4)) for i in primal_solution_ls]
        dual_solution_ls = w.tolist()
        results_dual_ls = [str(round(i, 4)) for i in dual_solution_ls]
        
        print("\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal primal solution is")
        print("\033[92m[ " + ', '.join(primal_variables_ls) + " ] = [ " + ', '.join(results_primal_ls) + " ]\033[0m")
        print("\nAnd the optimal dual solution is")
        print("\033[92m[ " + ', '.join(['w' + str(i + 1) for i in range(m)]) + " ] = [ " + ', '.join(results_dual_ls) + " ]\033[0m\n\n")
        
        return np.array(primal_solution_ls), w
        
    else:
        # Defining Dual of Restricted Primal Problem
        print("\n\033[94mDual of Restricted Primal Problem\033[0m")
        A_DRPP = np.concatenate((A_RPP.T, np.eye(len(variables_RPP), dtype = int)), axis = 1)
        b_DRPP = c_RPP
        c_DRPP = np.concatenate((-1 * b, np.zeros(len(variables_RPP), dtype = int)), axis = 0)
        artificial_variables_DRPP = ['x' + str(i) for i in range(no_variables + 1, no_variables + 2 * m + 1)]
        variables_DRPP = [variables[i] for i in range(no_variables) if i in Q] + artificial_variables_DRPP
        
        m_DRPP, n_DRPP = np.shape(A_DRPP)
        basic_indexes = [ind for ind in range(n_DRPP - m_DRPP, n_DRPP)]
        non_basic_indexes = [ind for ind in range(n_DRPP - m_DRPP)]
        tableau = construct_tableau(A_DRPP, b_DRPP, c_DRPP, basic_indexes)

        v = simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables_DRPP), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
        v = v[[index_w for index_w in range(m)]]
        
        flag_not_boundable = False
        if isinstance(v, bool) == True:
            flag_not_boundable = True
           
        
        if flag_not_boundable == False:
            print("\nv =", v)
            wa_minus_c = wA - c
            vA = np.ravel(v @ A)

            counter_flag = 0
            for i in range(no_variables):
                if vA[i] <= 0:
                    counter_flag += 1

            if counter_flag == no_variables:
                print("\n\033[1mOptimization process stopped :(\033[0m")
                print("\nThe Dual of RPP is unbounded and the primal is infeasible (vA <= 0)\n\n")

                return False, False

            else:
                quot_ls = [-1 * wa_minus_c[i] / vA[i] for i in range(no_variables) if vA[i] > 0]
                theta = min(quot_ls)
                print("\nTheta =", theta)

                w = w + theta * v
                print("\nw_updated =", w)

                return primal_dual(A, b, c, w, variables, itera + 1)
        else:
            print("\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe Dual of RPP is unbounded and the primal is infeasible\n\n")
            
            return False, False
        

To make possible the Primal Dual Method we are going to retake the functions of the Simplex Method in tableau format, which was made previously on other tasks.

In [4]:
def simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
        
    # Printing our initial parameters for this iteration
    #print("\n\n\033[1mIteration ", itera, "\033[0m")
    #print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
    #      + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZXN = np.ravel(tableau[0, [i + 1 for i in non_basic_indexes]])
    XBXN = tableau[1:, [i + 1 for i in non_basic_indexes]]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    #print("\nEntry tableau")
    #print(tableau)
    
    z_minus_c_max = max(ZXN)
    k = np.argmax(ZXN)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        #print("\n\n\033[1mOptimality reached\033[0m")
        #print("\nThe optimal BFS is")
        results_ls = [str(round(i, 4)) for i in b_]
        #print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        #print("\nWith performance z =", perf_z, "\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_solution = dict(sorted(dict_solution.items(), key = lambda x:x[0]))
        solution = np.array(list(sorted_dict_solution.values()))
        #print(solution)
        return solution
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(XBXN[:, k])
        
        #print("\nk =", k + 1, "-> column no.", k + 1, "of x_N part (", non_basic_variables[k], ")")
        #print("y_k")
        #print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(b_[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            #print("\nr =", r + 1, "-> column no.", r + 1, "of x_B part (", basic_variables[r], ")")
            #print("x_Br")
            #print(x_Br)
            
            # Pivoting
            pivot = y_k[r]
            #print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, non_basic_indexes[k] + 1] * tableau[r + 1, :]
            
            #print("\nPivoted tableau")
            #print(tableau)
            # Exchanging the indexes
            #print("\n\033[94m", non_basic_variables[k], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k]
            non_basic_indexes[k] = aux_vars
            # Exchanging columns in tableau
            #aux_column = tableau[:, m + k + 1]
            #tableau[:, m + k + 1] =  tableau[:, 1 + r]
            #tableau[:, 1 + r] = aux_column
            
            # Recursive call
            return simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera + 1)

        else:   # Not boundable
            
            #print("\n\n\033[1mOptimization process stopped :(\033[0m")
            #print("\nThe optimal BFS is not boundable\n\n")
            
            return False
            

In addition, we are going to retake also the Dual Simplex Method to verify some results.

In [5]:
def dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZX = np.ravel(tableau[0, 1:n_plus_m + 1])
    XBX = tableau[1:, 1:n_plus_m + 1]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    positive_b = True
    for b_i in np.ravel(XBRHS):
        if b_i < 0:
            positive_b = False
            break
    
    # Validating if we stop or not the optimization
    if positive_b:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal primal solution is")
        results_primal_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_primal_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z)
        print("\nAnd the optimal dual solution is")
        results_dual_ls = np.ravel(-1 * tableau[0, [i + 1 for i in slack_indexes]]).tolist()
        results_dual_ls_f = [str(round(i, 4)) for i in results_dual_ls]
        print("\033[92m[ " + ', '.join(['w' + str(i + 1) for i in range(m)]) + " ] = [ " + ', '.join(results_dual_ls_f) + " ]\033[0m\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_primal_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_primal_solution = dict(sorted(dict_primal_solution.items(), key = lambda x:x[0]))
        primal_solution = np.array(list(sorted_dict_primal_solution.values()))
        
        dict_dual_solution = dict(zip([i for i in range(m)], results_dual_ls))
        sorted_dict_dual_solution = dict(sorted(dict_dual_solution.items(), key = lambda x:x[0]))
        dual_solution = np.array(list(sorted_dict_dual_solution.values()))
        
        #print(solution)
        return primal_solution, dual_solution
        
    else:   # We continue
        
        br = min(XBRHS)
        r = np.argmin(XBRHS)
        
        # Calculating and printing the y_ri's
        y_r = np.ravel(XBX[r, :])
        
        print("\nr =", r + 1, "-> row no.", r + 1, "of x_B part (", basic_variables[r], ")")
        print("y_r")
        print(y_r)
        
        # Analyzing if the Dual Problem is or not boundable by the condition y_r >= 0
        flag = True
        aux_counter = 0
        for y_i in y_r:
            if y_i >= 0:
                aux_counter += 1      
        if aux_counter == len(y_r):
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_k_ls = list()
            for i in range(n_plus_m):
                if y_r[i] < 0:
                    quot_ls.append(ZX[i] / y_r[i])
                    index_k_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_k_arr = np.array(index_k_ls)
            k = index_k_arr[np.argmin(quot_arr)]
            min_quot = min(quot_arr)
            print("\nk =", k + 1, "-> column no.", k + 1, "of x part (", variables[k], ")")
            print("min_quotient")
            print(min_quot)
            
            # Pivoting
            pivot = y_r[k]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, k + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            k_relative = 0
            for i in range(len(non_basic_indexes)):
                if k == non_basic_indexes[i]:
                    k_relative = i
                    break
            print("\n\033[94m", non_basic_variables[k_relative], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k_relative]
            non_basic_indexes[k_relative] = aux_vars
            
            # Recursive call
            return dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe Dual Problem is not boundable, thus the primal feasible region is empty\n\n")
            
            return False, False
            

In [6]:
def construct_tableau(A, b, c, basic_indexes):
    m, n_plus_m = np.shape(A)
    B = A[:m, basic_indexes]
    c_b = c[basic_indexes]
    b_ = np.linalg.inv(B) @ b

    one_array = np.ones(1, dtype=int)
    zero_array_v = np.ravel(np.zeros(m, dtype=int)).reshape(1, m)
    ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
    XBX = np.linalg.inv(B) @ A
    ZRHS = np.full((1), c_b @ np.ravel(b_))
    XBRHS = b_.reshape(1, m)

    tableau = np.vstack((np.concatenate((one_array, ZX, ZRHS), axis = 0).reshape(1, n_plus_m + 2),
                  np.concatenate((zero_array_v.T, XBX, XBRHS.T), axis = 1)))
    
    return tableau

In [7]:
import itertools

def basic_indexes_dual_feasible_solution(A, b, c, indexes, m):
    all_combinations = itertools.combinations(indexes, m)
    possible_basic_indexes = list()
    for basic_indexes in all_combinations:
        try:
            basic_indexes = list(basic_indexes)
            B = A[:m, basic_indexes]
            c_b = c[basic_indexes]
            ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
            
            dual_feasible = True
            for zx in ZX:
                if zx > 0:
                    dual_feasible = False
                    break
            if dual_feasible:
                possible_basic_indexes.append(basic_indexes)  
        except:
            pass
    
    return possible_basic_indexes

In [8]:
def basic_variables_dual_feasible_solution(possible_basic_indexes, variables):
    possible_basic_variables = list()
    for psb in possible_basic_indexes:
        possible_basic_variables.append([variables[i_psb] for i_psb in psb])
    
    return possible_basic_variables

So, once we have already defined above functions we are able to apply it.

#### Example 1:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 3x_1 + 4x_2 + 6x_3 + 7x_4 + x_5 + 0x_6 + 0x_7$$
$$under \hspace{0.5cm} 2x_1 - x_2 + x_3 + 6x_4 - 5x_5 - x_6 + 0x_7 = 6$$
$$\hspace{1.4cm} x_1 + x_2 + 2x_3 + x_4 + 2x_5 + 0x_6 - x_7 = 3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0$$

On the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} 6w_1 + 3w_2$$
$$under \hspace{0.5cm} 2w_1 + w_2 \leq 3$$
$$\hspace{1.4cm} - w_1 + w_2 \leq 4$$
$$\hspace{1.4cm} w_1 + 2w_2 \leq 6$$
$$\hspace{1.4cm} 6w_1 + w_2 \leq 7$$
$$\hspace{1.4cm} -5w_1 + 2w_2 \leq 1$$
$$\hspace{1.4cm} -w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, we get the following elements:

\begin{equation} A = 
\left[
\begin{array}{ccccccc}
2 & -1 & 1 & 6 & -5 & -1 & 0\\
1 & 1 & 2 & 1 & 2 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
6\\
3\\
\end{array}
\right]
\end{equation}


\begin{equation} c =
\left[
\begin{array}{ccccccc}
3 & 4 & 6 & 7 & 1 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [9]:
A = np.matrix([[2, -1, 1, 6, -5, -1, 0], [1, 1, 2, 1, 2, 0, -1]])
b = np.array([6, 3])
c = np.array([3, 4, 6, 7, 1, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6", "x7"]

Besides that, we propose $w = [0, 0]$ as our initial dual feasible solution that will enable us to start the Primal Dual Method.

In [10]:
w = np.array([0, 0])

In [11]:
primal_dual(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(w), copy.deepcopy(variables))



Iteration  1 

Q = [6, 7]

Restricted Primal Problem

Variables RPP
['x6', 'x7', 'x8', 'x9']

Matrix A RPP
[[-1  0  1  0]
 [ 0 -1  0  1]]

Vector c RPP
[0 0 1 1]

Solution RPP
[0. 0. 6. 3.]

Performance RPP
9.0

Dual of Restricted Primal Problem
[1. 1. 1. 1. 0. 0.]

v = [1. 1.]

Theta = 1.0

w_updated = [1. 1.]


Iteration  2 

Q = [1, 4]

Restricted Primal Problem

Variables RPP
['x1', 'x4', 'x8', 'x9']

Matrix A RPP
[[2 6 1 0]
 [1 1 0 1]]

Vector c RPP
[0 0 1 1]

Solution RPP
[3. 0. 0. 0.]

Performance RPP
0.0

Optimality reached

The optimal primal solution is
[ x1, x2, x3, x4, x5, x6, x7 ] = [ 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0 ]

And the optimal dual solution is
[ w1, w2 ] = [ 1.0, 1.0 ]




(array([3., 0., 0., 0., 0., 0., 0.]), array([1., 1.]))

#### Exercise 6.43:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 9x_1 + 7x_2 + 4x_3 + 2x_4 + 6x_5 + 10x_6$$
$$under \hspace{0.5cm} x_1 + x_2 + x_3 + x_4 + x_5 + x_6 = 13$$
$$\hspace{1.4cm} x_1 + 0x_2 + 0x_3 + x_4 + 0x_5 + 0x_6 = 6$$
$$\hspace{1.4cm} 0x_1 + x_2 + 0x_3 + 0x_4 + x_5 + 0x_6 = 4$$
$$\hspace{1.4cm} 0x_1 + 0x_2 + x_3 + 0x_4 + 0x_5 + x_6 = 3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

On the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} 8w_1 + 5w_2 + 6w_3 + 4w_4 + 3w_5$$
$$under \hspace{0.5cm} w_1 + w_2 \leq 9$$
$$\hspace{1.4cm} w_1 + w_3 \leq 7$$
$$\hspace{1.4cm} w_1 + w_4 \leq 4$$
$$\hspace{1.4cm} w_1 + w_2 \leq 2$$
$$\hspace{1.4cm} w_1 + w_3 \leq 6$$
$$\hspace{1.4cm} w_1 + w_4 \leq 10$$
$$\hspace{1cm}w_1, w_2, w_3, w_4 \hspace{0.5cm}unrestricted$$

Then, we get the following elements:

\begin{equation} A = 
\left[
\begin{array}{cccccc}
1 & 1 & 1 & 1 & 1 & 1\\
1 & 0 & 0 & 1 & 0 & 0\\
0 & 1 & 0 & 0 & 1 & 0\\
0 & 0 & 1 & 0 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
13\\
6\\
4\\
3\\
\end{array}
\right]
\end{equation}


\begin{equation} c =
\left[
\begin{array}{ccccccc}
9 & 7 & 4 & 2 & 6 & 10\\
\end{array}
\right]
\end{equation}

In [36]:
A = np.matrix([[1, 1, 1, 1, 1, 1], 
               [1, 0, 0, 1, 0, 0],
               [0, 1, 0, 0, 1, 0], 
               [0, 0, 1, 0, 0, 1]])
b = np.array([13, 6, 4, 3])
c = np.array([9, 7, 4, 2, 6, 10])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

Besides that, we propose $w = [0, 1, 1, 5, 0]$ as our initial dual feasible solution that will enable us to start the Primal Dual Method.

In [37]:
w = np.array([4, 5, 3, 0])

In [38]:
primal_dual(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(w), copy.deepcopy(variables))



Iteration  1 

Q = [1, 2, 3]

Restricted Primal Problem

Variables RPP
['x1', 'x2', 'x3', 'x7', 'x8', 'x9', 'x10']

Matrix A RPP
[[1 1 1 1 0 0 0]
 [1 0 0 0 1 0 0]
 [0 1 0 0 0 1 0]
 [0 0 1 0 0 0 1]]

Vector c RPP
[0 0 0 1 1 1 1]

Solution RPP
[6. 4. 3. 0. 0. 0. 0.]

Performance RPP
0.0

Optimality reached

The optimal primal solution is
[ x1, x2, x3, x4, x5, x6, x7 ] = [ 6.0, 4.0, 3.0, 0.0, 0.0, 0.0, 0.0 ]

And the optimal dual solution is
[ w1, w2, w3, w4 ] = [ 4, 5, 3, 0 ]




(array([6., 4., 3., 0., 0., 0., 0.]), array([4, 5, 3, 0]))

#### Exercise 6.44:
Solve the next linear programming problem:

$$min \hspace{0.5cm} x_1 + 0x_2 + 2x_3 - x_4$$
$$under \hspace{0.5cm} x_1 + x_2 + x_3 + x_4 \leq 6$$
$$\hspace{1.4cm} 2x_1 - x_2 + 3x_3 - 3x_4 \geq 5$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

First, we introduce our slack variables $x_5, x_6$ and multiply by -1 the first constraint for mere convenience. So, the primal problem can be rewritten as

$$min \hspace{0.5cm} x_1 + 0x_2 + 2x_3 - x_4 + 0x_5 + 0x_6$$
$$under \hspace{0.5cm} - x_1 - x_2 - x_3 - x_4 - x_5 + 0x_6 = -6$$
$$\hspace{1.4cm} 2x_1 - x_2 + 3x_3 - 3x_4 + 0x_5 - x_6 = 5$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 6w_1 + 5w_2 $$
$$under \hspace{0.5cm} - w_1 + 2w_2 \leq 1$$
$$\hspace{1.4cm} - w_1 - w_2 \leq 0$$
$$\hspace{1.4cm} - w_1 + 3w_2 \leq 2$$
$$\hspace{1.4cm} - w_1 - 3w_2 \leq -1$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, we get the following elements:

\begin{equation} A = 
\left[
\begin{array}{cccccc}
-1 & -1 & -1 & -1 & -1 & 0\\
2 & -1 & 3 & -3 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-6\\
5\\
\end{array}
\right]
\end{equation}


\begin{equation} c =
\left[
\begin{array}{ccccccc}
1 & 0 & 2 & -1 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [15]:
A = np.matrix([[-1, -1, -1, -1, -1, 0],
               [2, -1, 3, -3, 0, -1]])
b = np.array([-6, 5])
c = np.array([1, 0, 2, -1, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

Besides that, we propose $w = [1, 1]$ as our initial dual feasible solution that will enable us to start the Primal Dual Method.

In [16]:
w = np.array([0, 0.5])

In [17]:
primal_dual(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(w), copy.deepcopy(variables))



Iteration  1 

Q = [1, 5]

Restricted Primal Problem

Variables RPP
['x1', 'x5', 'x7', 'x8']

Matrix A RPP
[[-1 -1  1  0]
 [ 2  0  0  1]]

Vector c RPP
[0 0 1 1]

Solution RPP
[ 2.5  0.  -3.5  0. ]

Performance RPP
-3.5

Dual of Restricted Primal Problem
[0. 0. 0. 0. 1. 1.]

v = [0. 0.]

Optimization process stopped :(

The Dual of RPP is unbounded and the primal is infeasible (vA <= 0)




(False, False)

Using Dual Simplex Method we get...

In [18]:
A = np.matrix([[-1, -1, -1, -1, -1, 0],
               [2, -1, 3, -3, 0, -1]])
b = np.array([-6, 5])
c = np.array([1, 0, 2, -1, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [19]:
basic_indexes = [4, 5]
non_basic_indexes = [0, 1, 2, 3]
slack_indexes = [4, 5]

In [20]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [21]:
tableau

matrix([[ 1., -1.,  0., -2.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  1.,  1.,  1.,  0.,  6.],
        [ 0., -2.,  1., -3.,  3.,  0.,  1., -5.]])

In [22]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[['x1', 'x3'], ['x1', 'x5'], ['x4', 'x5'], ['x4', 'x6']]

In [23]:
basic_indexes = [3, 5]
non_basic_indexes = [0, 1, 2, 4]
slack_indexes = [4, 5]

In [24]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [25]:
tableau

matrix([[  1.,  -2.,  -1.,  -3.,   0.,  -1.,   0.,  -6.],
        [  0.,   1.,   1.,   1.,   1.,   1.,   0.,   6.],
        [  0.,  -5.,  -2.,  -6.,   0.,  -3.,   1., -23.]])

In [26]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x4, x6, x1, x2, x3, x5 ]


Entry tableau
[[  1.  -2.  -1.  -3.   0.  -1.   0.  -6.]
 [  0.   1.   1.   1.   1.   1.   0.   6.]
 [  0.  -5.  -2.  -6.   0.  -3.   1. -23.]]

r = 2 -> row no. 2 of x_B part ( x6 )
y_r
[-5. -2. -6.  0. -3.  1.]

k = 5 -> column no. 5 of x part ( x5 )
min_quotient
0.3333333333333333

pivot = -3.0

Pivoted tableau
[[ 1.     -0.3333 -0.3333 -1.      0.      0.     -0.3333  1.6667]
 [ 0.     -0.6667  0.3333 -1.      1.      0.      0.3333 -1.6667]
 [-0.      1.6667  0.6667  2.     -0.      1.     -0.3333  7.6667]]

 x5 enters and  x6 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x4, x5, x1, x2, x3, x6 ]


Entry tableau
[[ 1.     -0.3333 -0.3333 -1.      0.      0.     -0.3333  1.6667]
 [ 0.     -0.6667  0.3333 -1.      1.      0.      0.3333 -1.6667]
 [-0.      1.6667  0.6667  2.     -0.      1.     -0.3333  7.6667]]

r = 1 -> row no. 1 of x_B part ( x4 )
y_r
[-0.6667  0.3333 -1.      1.      0.      0.3333]

k = 1 -> co

(array([2.5, 0. , 0. , 0. , 3.5, 0. ]), array([0. , 0.5]))

#### Exercise 6.47:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 7x_1 + 2x_2 + x_3 + 4x_4 + 6x_5$$
$$under \hspace{0.5cm} 3x_1 + 5x_2 - 6x_3 + 2x_4 + 4x_5 = 27$$
$$\hspace{1.4cm} x_1 + 2x_2 + 3x_3 - 7x_4 + 6x_5 \geq 2$$
$$\hspace{1.4cm} 9x_1 - 4x_2 + 2x_3 + 5x_4 - 2x_5 = 16$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

First, we introduce our slack variable $x_6$, then multiply by -1 the objective function in order to turn the maximization problem into a minimization one. So, the primal problem can be rewritten as

$$min \hspace{0.5cm} -7x_1 - 2x_2 - x_3 - 4x_4 - 6x_5$$
$$under \hspace{0.5cm} 3x_1 + 5x_2 - 6x_3 + 2x_4 + 4x_5 + 0x_6= 27$$
$$\hspace{1.4cm} x_1 + 2x_2 + 3x_3 - 7x_4 + 6x_5 - x_6 = 2$$
$$\hspace{1.4cm} 9x_1 - 4x_2 + 2x_3 + 5x_4 - 2x_5 + 0x_6 = 16$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} 27w_1 + 2w_2 + 16w_3 $$
$$under \hspace{0.5cm} 3w_1 + w_2 + 9w_3 \leq -7$$
$$\hspace{1.4cm} 5w_1 + 2w_2 - 4w_3 \leq -2$$
$$\hspace{1.4cm} - 6w_1 + 3w_2 + 2w_3 \leq -1$$
$$\hspace{1.4cm} 2w_1 - 7w_2 + 5w_3 \leq -4$$
$$\hspace{1.4cm} 4w_1 + 6w_2 - 2w_3 \leq -6$$
$$\hspace{1.4cm} 0w_1 - w_2 + 0w_3 \leq 0$$
$$\hspace{1cm}w_1, w_2, w_3 \hspace{0.5cm}unrestricted$$

First, we have to check if the feasible region for the dual problem exists or not.

Fixing w_2 and w_3 as zero we get
$$w_1\leq \frac{-7}{3} \land w_1 \leq \frac{-2}{5} \land w_1 \geq \frac{1}{6} \land w_1 \leq -2 \land w_1 \leq \frac{-3}{2} = \emptyset$$

Thus, the dual feasible region is empty and the primal problem is unbounded or infeasible.


For mere validation, we are going to try to solve the problem by the simplex method in tableau format...

In [27]:
A = np.matrix([[3, 5, -6, 2, 4, 0],
               [1, 2, 3, -7, 6, -1],
               [9, -4, 2, 5, -2, 0]])
b = np.array([27, 2, 16])
c = np.array([-7, -2, -1, -4, -6, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [28]:
basic_indexes = [3, 4, 5]
non_basic_indexes = [0, 1, 2]

In [29]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [30]:
tableau

matrix([[  1.    ,   0.75  ,  -5.25  ,  10.1667,   0.    ,   0.    ,
           0.    , -45.4167],
        [  0.    ,   1.75  ,  -0.25  ,  -0.1667,   1.    ,   0.    ,
           0.    ,   4.9167],
        [  0.    ,  -0.125 ,   1.375 ,  -1.4167,  -0.    ,   1.    ,
           0.    ,   4.2917],
        [  0.    , -14.    ,   8.    , -10.3333,  -0.    ,  -0.    ,
           1.    , -10.6667]])

In [31]:
simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))

False

In [32]:
possible_basic_indexes = basic_indexes_dual_feasible_solution(A, b, c, [i for i in range(len(variables))], np.shape(A)[0])
possible_basic_variables = basic_variables_dual_feasible_solution(possible_basic_indexes, variables)
possible_basic_variables

[]